In [1]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

jpath=os.path.dirname(colabexts.__file__)
jcom = f'{jpath}/jcommon.ipynb'
%run $jcom

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)


In [10]:
%%writefile "/opt/utils/geo_utils/update_version.py"
#!/usr/bin/env python
# GENERATED FROM geoapp/notebooks/commandLineUtils
import re, sys, colabexts.utils as colabexts_utils

def upver(file = "../tseries/templates/tseries/footer.html"):
    txt = open(file, "r").read()
    comp = re.compile("(.*version:[ ]+)([\d\.]+)",  flags=re.MULTILINE|re.DOTALL|re.IGNORECASE)
    vn=re.findall(comp, txt)
    if (vn):
        vn = vn[0][1]
        nv = vn.split(".")
        nv[-1] = str(int(nv[-1])+1 )
        ns = ".".join(nv)
        result = re.sub(comp, rf"\1 {ns}", txt)
        print(f"Updated {txt[-256:]}\n{vn}==>{ns}\n{result[-256:]}")
        open(file, "w").write(result)
    else:
        print("Pattern 'version: x.x.x' not found")
    
#----------------------------------------------------------------------------------
if __name__ == '__main__' and not colabexts_utils.inJupyter():
    try:
        upver(sys.argv[1])
    except:
        print("Require filename")

Overwriting /opt/utils/geo_utils/update_version.py


# Check URL and send email and fix if it is borken

Put this in your crontab entry to run it every 5 minutes; you may check the output using tail

[ ] crontab -e
*/5 * * * * * ../gen/checksite.py > /tmp/GOTIT

[ ] tail -f /tmp/GOTIT

~

In [1]:
%%writefile "../gen/checksite.py"
#!/usr/local/bin/python

import datetime, requests, time, sys, os
from django.core.mail import send_mail
from django.conf import settings
sys.path.append(os.path.expanduser("~/.django") )
import my_config

if ( not settings.configured):
    settings.configure(
        EMAIL_USE_TLS       = my_config.EMAIL_USE_TLS       ,
        EMAIL_HOST          = my_config.EMAIL_HOST          ,
        EMAIL_PORT          = my_config.EMAIL_PORT          ,
        DEFAULT_FROM_EMAIL  = my_config.DEFAULT_FROM_EMAIL  ,
        EMAIL_HOST_USER     = my_config.EMAIL_HOST_USER     ,
        EMAIL_HOST_PASSWORD = my_config.EMAIL_HOST_PASSWORD ,
        EMAIL_BACKEND       = 'django.core.mail.backends.smtp.EmailBackend'
    )

def sendMail(to="sada@geospaces.org", subject="hello", msg="Check server!"):
    r = [f.strip() for f in to.split(",")]

    try:
        ret = send_mail( from_email= settings.DEFAULT_FROM_EMAIL, 
                        subject=subject, message=msg, 
                        recipient_list=r,fail_silently = False)
    except Exception as e:
        print("ERROR: ", e)

    print(f'Sending Emails to {r} {ret}')
    
    
def fixsite(s=""):
    print(f"+++++ Fixing Site: {s}")
    SSHC = os.path.expanduser("~/ec2-keys/du1.sh")
    out=subprocess.run([SSHC, "start.sh"],  capture_output=True)
    o=out.stdout.decode("utf-8")
    
    out=subprocess.run([cmd1, "screen -ls"],  capture_output=True)
    o=out.stdout.decode("utf-8")
    print("+++++ Fixing ", o)

    
def main(site="https://www.geospaces.org"):
    r, i = None, 0
    print( f"?: {datetime.datetime.now()}:  ", end="")
    try:
        r = requests.get(site)
        if ( r.status_code == 200):
            txt = {r.text[0:60].replace('\n', '<NL>')}
            print(f"+ {r.status_code} : {txt} ...")
    except Exception as e:
        print( f"ERROR: {e}")
        r = None
        
    if ( r is None):
        print(f"ERROR:  Fixing ...")
        sendMail("sada@geospaces.org", f"{site} Down", f"Checking {site} ")
        fixsite(s)
    
#----------------------------------------------------------------------------------
def inJupyter():
    try:    get_ipython; return True
    except: return False
    
if __name__ == '__main__' and not inJupyter():
    main("https://www.geospaces.org")
    pass


Writing ../gen/checksite.py


# Send multiple request and see the timing

In [1]:
#%%writefile "/opt/utils/geo_utils/mrequest.py"
#!/usr/bin/env python

import datetime, requests, time, sys, os, time
import multiprocessing

'''
Some notes on using multiprocessing 

Ex1: Creating one off background process - expect no returns

    files = "a1.csv a2.csv a3.csv".split()
    for f in files:
        p = multiprocessing.Process(target=read_parq, args=([f, 10] ))
        p.start()

Ex2: Want return values? Create pool

    files = "a1.csv a2.csv a3.csv".split()
    nthreads = min(os.cpu_count(), len(files))
    pool = Pool(processes=nthreads)

    dfs=[]
    for result in pool.starmap(read_parq, [(f, 5, 1) for f in files[0:2]]):
        dfs.append(result)
'''
def testURL(site="https://www.geospaces.org", id=0):
    r, t1 = None, datetime.datetime.now()
    print( f"ID:=> {id} ?: {t1}:  ", end="")
    try:
        r = requests.get(site)
    except Exception as e:
        print( f"ERROR: {e}")
        return
    
    t2 = datetime.datetime.now()
    if ( r is not None and r.status_code == 200):
        txt = {r.text[0:32].replace('\n', '<NL>')}
        print(f"ID:=> {id}+Took:{t2-t1}  {r.status_code} : {txt} ...", flush=True)
    else:
        print(f"ID:=> {id}+ ERROR: {t2-t1} Status: {r.status_code}", flush=True)
#-----------------------------------------------
def main(*argv):  
    if ( len(argv) <2):
        print(f"Usage: <url> [#of times to call: default is 1] {len(argv)} {argv}")
        return

    n, url, delay = 1, argv[1], 0
    if ( len(argv) >= 3):
        n  = int(argv[2])
    if ( len(argv) >= 4):
        delay  = int(argv[3])
    #ps = []
    print(f"Testing URL: {url} #times: {n} Delay:{delay}")
    for i in range(n):
        p = multiprocessing.Process(target=testURL, args=([url, i] ))
        r = p.start()
        if ( delay > 0):
            print(f"Sleeping for {delay} seconds")
            time.sleep(delay)

def inJupyter():
    try:    get_ipython; return True
    except: return False

if __name__ == '__main__' and not inJupyter():
    main(*sys.argv)
    pass


#main("","https://www.linkedin.com/",1)

In [51]:
testURL("https://www.linkedin.com/")
#main("","https://www.linkedin.com/",1)

ID:=> 0 ?: 2023-08-06 18:41:38.182238:  ID:=> 0+Took:0:00:00.838916  200 : {'<!DOCTYPE html><NL><NL><NL>    <NL>    <NL>    '} ...


# Tamper Monkey

In [ ]:
# Tamper Monkey

Create the following script

```
// ==UserScript==
// @name         sadatest
// @namespace    http://tampermonkey.net/
// @version      0.1
// @description  try to take over the world!
// @author       You
// @match        https://learning.oreilly.com/library/view/*
// @grant        GM_registerMenuCommand
// @run-at       context-menu
// ==/UserScript==
(function() {
    'use strict';
    // Your code here...
    console.log("hello");
    hide()
})();
GM_registerMenuCommand('Run my Script', function() {
   alert('it runs!');
})
function test2(clsname){
    var ae=document.getElementsByClassName(clsname);
    var lengthOfArray=ae.length;

    for (var i=0; i<lengthOfArray;i++){
        ae[i].style.display='none';
    }
    return ae;
}
function test21(clsname){
    var ae=document.getElementById(clsname);
    ae.style.maxWidth="1800px";
    ae.style.margin=0;
    ae.style.padding=0;
    return ae
}
function hide(){
    test2('topbar')
    test2('topbar t-topbar')
    test2('Index')
    test2('pagefoot')
    test2('interface-controls')
    var ae = test21('sbo-rt-content')

    return ae
}
```


# CONNECT TO WEBSOCKET

In [10]:
import requests

url="http://localhost:7004/broadcast/?room=alert&message=test how are you 1"
r = requests.post(url=url)


In [1]:
"EMAIL" in "DSDSKJ EMAIL AHJKDS"

True